In [1]:
import asdf
import numpy as np

# Introduction

ASDF files store their information using a tree structure. This allows the stored information be be
hierarchically organized within the file. Without any extensions, this tree is a nested combination
of basic data structures: maps, lists, arrays, strings, and numbers. In Python these types correspond
to `dict`, `list`, `np.ndarray`, `str`, and `int`, `float`, `complex`. Where `np.ndarray` is treated in
a special way. These data types can be extended to other types and objects using extensions.

In the ASDF python library, this tree can be created using a Python dictionary using key/value pairs.
Indeed, one can interact with any `AsdfFile` tree as if it were a dictionary. Note that due to limits
imposed by Python, keys must be `bool`, `int`, or `str` types only, while data information can be any
of the above data types.

# Creating ASDF files using basic python types

Lets first create an ASDF file with the key/value pair `"hello": "world"`:

In [2]:
tree = {"hello": "world"}
ff = asdf.AsdfFile(tree)
ff.write_to("hello.asdf")
ff["hello"]

'world'

Open the `hello.asdf` file in your favorite text editor. You should see a something that looks like:

In [3]:
with open("hello.asdf", "r") as f:
    print(f.read())

#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 2.12.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension.BuiltinExtension
    software: !core/software-1.0.0 {name: asdf, version: 2.12.0}
hello: world
...



Notice that the file contains more information than just the `"hello": "world"` key value that we
entered. It contains information on the library used to create the file under `asdf_library`, and
information on what the ASDF library needs (schemas, extensions, etc.) to deserialize the stored 
data under `history`. 

Next lets create a file that stores information using all the other basic python types
(avoiding arrays for now):

In [4]:
tree = {
    "hello": "world",
    "foo": 42,
    "bar": 3.14,
    "imaginary": complex(2, 3),
    "animals": ["cat", "dog", "bird"],
    "data": {"mean": 3.14, "std": 2.71},
}
ff = asdf.AsdfFile(tree)
ff.write_to("example.asdf")

Now open `example.asdf` in your text editor. You should see something like:

In [5]:
with open("example.asdf", "r") as f:
    print(f.read())

#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 2.12.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension.BuiltinExtension
    software: !core/software-1.0.0 {name: asdf, version: 2.12.0}
animals: [cat, dog, bird]
bar: 3.14
data: {mean: 3.14, std: 2.71}
foo: 42
hello: world
imaginary: !core/complex-1.0.0 (2+3j)
...



Again observe that all the data we added to our tree is contained within our asdf file. Notice in
particular the `imaginary` data now has a yaml tag denoting that the data is a complex number, this
tag will be used by ASDF to correctly deserialize this data as a `complex` type later.

## Updating your ASDF files

Recall that opening ASDF files can be simply done with the `asdf.open` command much like the standard
Python `open` command, with the exception that one must explicitly use the `mode` keyword to specify
the open method

For example:

In [6]:
with asdf.open("example.asdf") as af:
    print(f'{af["hello"]=}')
    print(f'{af["foo"]=}')
    print(f'{af["bar"]=}')
    print(f'{af["imaginary"]=}')
    print(f'{af["animals"]=}')
    print(f'{af["data"]=}')

af["hello"]='world'
af["foo"]=42
af["bar"]=3.14
af["imaginary"]=(2+3j)
af["animals"]=['cat', 'dog', 'bird']
af["data"]={'mean': 3.14, 'std': 2.71}


ASDF files can also be updated in the in a similar way. This time by opening the
file as writable and calling the `update` method:

In [7]:
with asdf.open("example.asdf", mode="rw") as af:
    af["new"] = "cool new stuff"
    af.update()

In [8]:
with open("example.asdf", "r") as f:
    print(f.read())

#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 2.12.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension.BuiltinExtension
    software: !core/software-1.0.0 {name: asdf, version: 2.12.0}
animals: [cat, dog, bird]
bar: 3.14
data: {mean: 3.14, std: 2.71}
foo: 42
hello: world
imaginary: !core/complex-1.0.0 (2+3j)
new: cool new stuff
...



# Creating ASDF files with numpy arrays

Beyond the maps, lists, strings, and numbers built into Python, ASDF can save arrays, in particular
numpy arrays (`nd.array`). Indeed, much of ASDF is dedicated to efficiently saving arrays.

For example if suppose we want to save a random 8x8 numpy array:

In [9]:
tree = {"random_array": np.random.rand(8, 8)}
ff = asdf.AsdfFile(tree)
ff.write_to("random.asdf")

Now opening this file in your text editor will result in something like:

In [10]:
with open("random.asdf", "r", encoding="unicode_escape") as f:
    print(f.read())

#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 2.12.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension.BuiltinExtension
    software: !core/software-1.0.0 {name: asdf, version: 2.12.0}
random_array: !core/ndarray-1.0.0
  source: 0
  datatype: float64
  byteorder: little
  shape: [8, 8]
...
ÓBLK 0                             ÔQÞÑ¬h¬ÄP#X,¨h ö#\8¨ðí?8­þFRá?¶ë¸Ö>ãÕ?×LÏë?\NÍðÉá?_LM\â?SQÍuºà?÷µ±¦è?zj ÅR¹ä?è¬ïYè?(8¤ÒÏ?Ôz:çä? ¸(,Þ? L×o´Å?ü9ÚìýJÒ?bOß?æz	ÿNHÙ?SÜ`#½ë?¬LNÌ¦á?À4Æ Ñ?,ë¸¼ß? È¹/rå?@WûG7x?G# Ü?ÐÀà®Zºç?W|prë?êSà?­É~ã?x¯0aÃ|î?Vé#ù,cà?¹
FFî?k»^Eê?öPSO8îà?I^_¬Æ?2k±ré?xã¸­Ã?4E¸ìÖÔ?Kc$«C­ä?ÌÐÁ	EÌ?xÙ?Ä?øÓJ2¥é?ä¼(mÍÎ?Ne÷DJ«á?l/v¸·É?XK.·û½?Ðÿ÷««?O=Zò%ì?(,ÞÈ?qSõ;®Ê?Á¨mR{ä?à/ÿâH?¨*Á](À?Ì5.SøÄ?{i2ÎV

Observe that at the end of the file that there is apparently some binary data. This binary data contains the information
in the random array we wrote. Indeed, when ASDF writes arrays to the file it stores them as binary data in a block after
the YAML section of the file rather in the section itself. Note that `random_array` in the YAML section stores some
information about the nature of the array and includes the `source` key. This `source` value references which binary block 
(in this case block `0`) the data is stored in.

Indeed if we update the file with another array to the file we get a second block:

In [11]:
with asdf.open("random.asdf", mode="rw") as ff:
    ff.tree.update({"new_array": np.random.rand(10, 10)})
    ff.update()

Opening `random.asdf` in your text editor will give something like:

In [12]:
with open("random.asdf", "r", encoding="unicode_escape") as f:
    print(f.read())

#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 2.12.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension.BuiltinExtension
    software: !core/software-1.0.0 {name: asdf, version: 2.12.0}
new_array: !core/ndarray-1.0.0
  source: 1
  datatype: float64
  byteorder: little
  shape: [10, 10]
random_array: !core/ndarray-1.0.0
  source: 0
  datatype: float64
  byteorder: little
  shape: [8, 8]
...
ÓBLK 0                             ÔQÞÑ¬h¬ÄP#X,¨h ö#\8¨ðí?8­þFRá?¶ë¸Ö>ãÕ?×LÏë?\NÍðÉá?_LM\â?SQÍuºà?÷µ±¦è?zj ÅR¹ä?è¬ïYè?(8¤ÒÏ?Ôz:çä? ¸(,Þ? L×o´Å?ü9ÚìýJÒ?bOß?æz	ÿNHÙ?SÜ`#½ë?¬LNÌ¦á?À4Æ Ñ?,ë¸¼ß? È¹/rå?@WûG7x?G# Ü?ÐÀà®Zºç?W|prë?êSà?­É~ã?x¯0aÃ|î?Vé#ù,cà?¹
FFî?k»^Eê?öPSO8îà?I^_¬Æ?2k±ré?xã¸­Ã?4E¸ìÖÔ?Kc$«C­ä?ÌÐÁ	EÌ?xÙ?Ä?øÓJ2¥é?

This now has a `new_array` key, which contains a second `source: 1` meaning there is a second binary block.

Now observe that ASDF is smart about storing arrays as binary data; meaning that, if arrays are shared between
entries in the tree, the same binary block is used. Indeed, this extends to sharing views on the data:

In [13]:
duplicated_array = np.random.rand(8, 8)
multi_view_array = np.random.rand(8, 8)
tree = {
    "duplicated_array_0": duplicated_array,
    "duplicated_array_1": duplicated_array,
    "multi_view_array": multi_view_array,
    "new_view": multi_view_array[2:4, 3:6],
}
with asdf.open("random.asdf", mode="rw") as ff:
    ff.tree.update(tree)
    ff.update()

Opening `random.asdf` in your text editor once again gives something like:

In [14]:
with open("random.asdf", "r", encoding="unicode_escape") as f:
    print(f.read())

#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 2.12.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension.BuiltinExtension
    software: !core/software-1.0.0 {name: asdf, version: 2.12.0}
duplicated_array_0: &id001 !core/ndarray-1.0.0
  source: 3
  datatype: float64
  byteorder: little
  shape: [8, 8]
duplicated_array_1: *id001
multi_view_array: !core/ndarray-1.0.0
  source: 2
  datatype: float64
  byteorder: little
  shape: [8, 8]
new_array: !core/ndarray-1.0.0
  source: 0
  datatype: float64
  byteorder: little
  shape: [10, 10]
new_view: !core/ndarray-1.0.0
  source: 2
  datatype: float64
  byteorder: little
  shape: [2, 3]
  offset: 152
  strides: [64, 8]
random_array: !core/ndarray-1.0.0
  source: 1
  datatype: float64
  byteorder: little
  shape: [8, 8]
...
   

First, note that `duplicated_array_1` simply listed using a yaml anchor which points to `duplicated_array_0`. Second,
observe that the `source` in both `multi_view_array` and `new_array` are the same value (`2`) rather than distinct values.
So ASDF did not unnecessary duplicate the binary data.

# Serializing other objects

The ASDF library supports writing extensions for objects outside of the ASDF-standard, which we will explain
in detail in another lecture. Assuming that one has installed an ASDF extension to support some custom Python
objects, ASDF will be able to seamlessly save those objects with no additional effort.

For example, as part of the install for this course we installed the `asdf-astropy` package, which provides
extensions for writing many `astropy` objects.

In [15]:
import astropy

quantity = 50 * astropy.units.m
model = astropy.modeling.models.Gaussian2D(quantity, 2, 3, 4, 5)
time = astropy.time.Time("J2000")
coord = astropy.coordinates.ICRS(ra=1 * astropy.units.deg, dec=2 * astropy.units.deg)

tree = {
    "quantity": quantity,
    "model": model,
    "time": time,
    "coord": coord,
}

ff = asdf.AsdfFile(tree)
ff.write_to("astropy.asdf")

Notice that since `asdf-astropy` was installed, no extra effort was required to write these objects
even though they clearly fall outside the ASDF-standard objects we previously discussed.

Moreover examining `astropy.asdf` in your text editor results in:

In [16]:
with open("astropy.asdf", "r", encoding="unicode_escape") as f:
    print(f.read())

#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 2.12.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension.BuiltinExtension
    software: !core/software-1.0.0 {name: asdf, version: 2.12.0}
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension._manifest.ManifestExtension
    extension_uri: asdf://asdf-format.org/astronomy/coordinates/extensions/coordinates-1.0.0
    software: !core/software-1.0.0 {name: asdf-astropy, version: 0.2.1}
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension._manifest.ManifestExtension
    extension_uri: asdf://asdf-format.org/core/extensions/core-1.5.0
    software: !core/software-1.0.0 {name: asdf-astropy, version: 0.2.1}
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension._manifest.Mani

Which clearly shows that we were successful in saving all the `astropy` objects we intended to.